# Consolidate Attendance lists

This notebook is for consolidating the attendancelists

In [2]:
# This reload library is just used for developing the REPUBLIC hOCR parser 
# and can be removed once this module is stable.
%reload_ext autoreload
%autoreload 2


#This is needed to add the repo dir to the path so jupyter
# can load the republic modules directly from the notebooks
import os
import sys
import re

# from collections import defaultdict
import pandas as pd
# import numpy as np
# #import xmltodict
# import networkx as nx
# from sqlalchemy import text


repo_dir = os.path.split(os.getcwd())[0]

if repo_dir not in sys.path:
    sys.path.append(repo_dir)


In [3]:
# let's repress the warnings, esp. from pandas
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

## Set up the attendancelists for one year

So we have something to work with

In [4]:
import run_attendancelist

In [5]:
runner1730 = run_attendancelist.RunAll(year=1730)


logging to attendancelist.log


In [95]:
abbr_delegates = runner1730.found_delegates
# runner1715.found_delegates
abbr_delegates['p_interval'] = abbr_delegates.apply(
    lambda x: pd.Interval(left=x["p_interval.left"], right=x["p_interval.right"]), axis=1)
# abbr_delegates['h_life'] = abbr_delegates.apply(
#     lambda x: pd.Interval(left=x["h_life.left"], right=x["h_life.right"]), axis=1)

In [96]:
runner1730.initial_find()

1. finding presidents
286 found
2.find provincial extraordinaris gedeputeerden


In [97]:
runner1730.gather_found_delegates()


3. finding unmarked text
4. joining presidents and delegates
total 56 found


In [ ]:
runner1730.verify_matches()

5. verifying matches
verifying spans
Empty DataFrame
Columns: [ref_id, geboortejaar, sterfjaar, colleges, functions, period, active_stgen, was_gedeputeerde, variants, name, found_in, id, sg, period_active.left, period_active.right, hypothetical_life.left, hypothetical_life.right, p_interval.left, p_interval.right, p_interval]
Index: []
session-1730-01-02-num-1 <republic.model.republic_attendancelist_models.MatchedText object at 0x7feb38068f70>
Empty DataFrame
Columns: [ref_id, geboortejaar, sterfjaar, colleges, functions, period, active_stgen, was_gedeputeerde, variants, name, found_in, id, sg, period_active.left, period_active.right, hypothetical_life.left, hypothetical_life.right, p_interval.left, p_interval.right, p_interval]
Index: []
session-1730-01-03-num-1 <republic.model.republic_attendancelist_models.MatchedText object at 0x7feb32a3e2b0>
Empty DataFrame
Columns: [ref_id, geboortejaar, sterfjaar, colleges, functions, period, active_stgen, was_gedeputeerde, variants, name, found

In [ ]:
searchobs = runner1730.searchobs
from IPython.display import HTML
htmlout=[]
for T in searchobs:
    ob = searchobs[T].matched_text
    url = searchobs[T].make_url()
    ob.mapcolors()
    rest = ob.serialize()
    rest = f"\n<h4>{T}</h4>\n" + rest
    if url:
        rest += f"""<br/><br/><a href='{url}'>link naar {T}-image</a><br/>"""
    htmlout.append(rest)
#out.reverse()
HTML("<br><hr><br>".join(htmlout))

In [79]:
c= Counter(run_attendancelist.make_groslijst(runner1715.searchobs))
tussenkeys = run_attendancelist.FuzzyKeywordGrouper(keyword_list=list(c.keys()))
tussenkeys.vars2graph()

[['-Brittannien', 'Brittannien'],
 ['-Vrieslandt',
  "-V'rieslandt",
  'ftVrieslandt',
  'stVrieslandt',
  'st-Vrieslandt',
  '-Vrrieslandt',
  'st-Vrrieslandt',
  'Vrieslandt'],
 ['rTsbrands',
  'Ysbrands',
  'Tsbrants',
  'Tebrands',
  '1sbrands',
  'Tsebrands',
  'Tabrands',
  'rsbrands',
  'Tubrands',
  '1sebrands',
  'Tsbrands',
  'rsebrands',
  'sbrands',
  'ssbrands',
  '15brants'],
 ['genomen', '2enomen', 'gen0men', '9enomen', 'bekomen', 'gegenomen'],
 ['Alphen', 'AlPhen'],
 ['Ambassadeur', 'Ambassadeurs'],
 ['Bamberg', 'Bambeesk', 'Bambeeck', 'Bambeek'],
 ['Broechhvysen',
  'Rroeckhuysen',
  'Broeckhuyfen',
  'Broeckhuysan',
  'Broeckkuysen',
  'Broeckbuysen',
  'Broekhuysen',
  'vanBroeckhuysen',
  'BroeckhuyJen',
  'Broeckhuysen',
  'Bratckhuysen',
  'Broechuysen',
  'Brocckhuysen',
  'Broeckhbuyses'],
 ['Castrieum', 'Castricum'],
 ['Coninck', 'Cininck', 'Cbninck', 'Cominck', 'ninck'],
 ['daer', 'der', 'DAE', 'WAEFR', 'WAER', 'haer', 'hair', 'WAXER', 'jaer'],
 ['des', 'DEE',

In [39]:
run_attendancelist.phrase_model.json

[{'phrase': 'BY refumptie gedelibereert zynde',
  'label': ['resolution_opening'],
  'custom': {'phrase': 'BY refumptie gedelibereert zynde',
   'label': 'resolution_opening',
   'max_offset': 4}},
 {'phrase': 'DE Resolutien, gisteren genomen',
  'variants': ['DE Resolutien gisteren ge-',
   'DE Resolutien voorleden vrijdagh ge-',
   'DE Resolutien voorleede ',
   'DE Resolutien eergisteren ge-',
   'DE Resolutien voorleden donderdag ge-',
   'DE Resolutien, eergisteren genomen',
   'DE Resolutien voorlede saterdagh ge-'],
  'label': ['resolution_summarized'],
  'custom': {'phrase': 'DE Resolutien, gisteren genomen',
   'label': 'resolution_summarized',
   'max_offset': 4,
   'variants': ['DE Resolutien, eergisteren genomen',
    'DE Resolutien voorleede ',
    'DE Resolutien gisteren ge-',
    'DE Resolutien eergisteren ge-',
    'DE Resolutien voorleden donderdag ge-',
    'DE Resolutien voorleden vrijdagh ge-',
    'DE Resolutien voorlede saterdagh ge-',
    'DE Resolutien gisteren 

Most likely the verification code needs an overhaul, but we'll see about that later.

## remediate text length

Sometimes, there are faulty attendancelists, that contain parts of the following resolutions. Further on, there is more finegrained code, but here we cutdown the size of the texts on basis of the mean text length, as there is not really an attendancelist that should be much longer than other.

## Calculate the ultimate delegate and cutoff texts beyond that

So we do not get spurious words in the resumption marked as delegates

In [23]:
import statistics
from collections import Counter
runner = runner1715
txtlens = [len(runner.searchobs[T].text) for T in runner.searchobs]
print(min(txtlens), max(txtlens))
print(statistics.mean(txtlens))
print(statistics.stdev(txtlens))
print(Counter(txtlens).most_common())

297 836
511.797385620915
112.62986585123504
[(750, 10), (468, 6), (456, 4), (486, 4), (545, 3), (432, 3), (478, 3), (467, 3), (537, 3), (475, 3), (444, 3), (501, 3), (567, 3), (546, 3), (417, 3), (379, 3), (508, 3), (458, 3), (392, 3), (422, 3), (469, 3), (612, 2), (609, 2), (745, 2), (632, 2), (690, 2), (616, 2), (598, 2), (582, 2), (790, 2), (447, 2), (531, 2), (530, 2), (520, 2), (471, 2), (642, 2), (560, 2), (438, 2), (510, 2), (519, 2), (407, 2), (459, 2), (400, 2), (419, 2), (429, 2), (405, 2), (512, 2), (440, 2), (481, 2), (441, 2), (418, 2), (495, 2), (527, 2), (396, 2), (509, 2), (489, 2), (443, 2), (507, 2), (404, 2), (411, 2), (493, 2), (470, 2), (543, 2), (373, 2), (397, 2), (457, 2), (505, 2), (535, 2), (538, 2), (327, 1), (415, 1), (727, 1), (742, 1), (687, 1), (744, 1), (796, 1), (686, 1), (651, 1), (608, 1), (707, 1), (752, 1), (729, 1), (679, 1), (621, 1), (606, 1), (605, 1), (709, 1), (653, 1), (712, 1), (753, 1), (787, 1), (716, 1), (453, 1), (577, 1), (476, 1), (656

In [382]:
from republic.model.republic_phrase_model import resolution_phrase_model
resolution_phrase_model[-3:]
resumption = resolution_phrase_model[-3:]

[{'phrase': 'BY refumptie gedelibereert zynde',
  'label': 'resolution_opening',
  'max_offset': 4},
 {'phrase': 'DE Resolutien, gisteren genomen',
  'label': 'resolution_summarized',
  'max_offset': 4,
  'variants': ['DE Resolutien, eergisteren genomen',
   'DE Resolutien voorleede ']},
 {'phrase': 'zyn gelesen en gerefumeert', 'label': 'resolution_summarized'}]

In [392]:
yesterdays = ['gisteren', 'eergisteren', 'voorleden donderdag',
              'voorleden vrijdagh', 'voorlede saterdagh']
searchstring = "DE Resolutien {g} ge-"
for g in yesterdays:
    samenv.append(searchstring.format(g=g))
resumption[1]["variants"].extend(samenv)

In [393]:
from fuzzy_search.fuzzy_phrase_searcher import FuzzyPhraseSearcher
from fuzzy_search.fuzzy_phrase_model import PhraseModel

# highger matching thresholds for higher quality OCR/HTR (higher precision, recall should be good anyway)
# lower matching thresholds for lower quality OCR/HTR (higher recall, as that's the main problem)
config = {
    "char_match_threshold": 0.8,
    "ngram_threshold": 0.6,
    "levenshtein_threshold": 0.8,
    "ignorecase": False,
    "ngram_size": 2,
    "skip_size": 2,
}

# initialize a new searcher instance with the config
resumption_searcher = FuzzyPhraseSearcher(config)

# create a list of domain keywords and phrases
#variants = [{'phrase': k, 'variants': v['variants']} for k, v in runner1715.all_matched.items()]
phrase = "DE Reslolutien, gisteren genomen, zijn gelesen ende geresumeert, gelijck"

# create a PhraseModel object from the domain phrases
phrase_model = PhraseModel(resumption)
# phrase_model.add_variants(variants)
resumption_searcher.index_phrase_model(phrase_model=phrase_model)

In [395]:
from scipy.special import softmax
from numpy import argmax
def score_match(match):
    result  = sum([match.levenshtein_similarity, match.character_overlap, match.ngram_overlap, score_levenshtein_distance_ratio(str(match.variant.exact_string),str(match.phrase.exact_string))])
    return result



In [396]:
res

[PhraseMatch(phrase: "DE Resolutien, gisteren genomen", variant: "DE Resolutien, gisteren genomen",string: "DE Resolutien, gisteren genomen", offset: 342)]

In [ ]:
for T in runner.searchobs:
    ob = runner.searchobs[T]
    res = resumption_searcher.find_matches(ob.text, include_variants=True, use_word_boundaries=False)
    if res:
        softscore = softmax([score_match(match) for match in res])
        i = argmax(softscore)
        res[i].offset

now we try to truncate the text after the last delegate. The delegates in the attendance lists appear in a fixed order of provinces. As there were a rotating, but limited number of delegates for the provinces at any given time, there is an even smaller number of delegates who appear as the last delegate in the attendancelist. For the 1715 year for instance the most common last delegate is Van der Lelij, who appeared 254 times, almost 32 times the 8 appearence of the next frequent delegate (du Tour). 

We check the attendance lists for inconsistencies. Even if the delegate for the last province (Friesland - English *Frysia*) was not present, or if the matcher missed him this will still truncate spurious delegate markers.


In [316]:
from collections import Counter
lastfragments = []
lastpatterns = []
for ob in runner1715.searchobs:
    lastknownfragment = ''
    lastpattern = ''
    fragments = runner1715.searchobs[ob].get_fragments()
    fragments.reverse()
    for item in fragments:
        f = item.get('delegate_name') or ''
        p = item.get('pattern') or ''
        if  f != '':
            lastknownfragment = item['delegate_name']
            lastfragments.append(lastknownfragment)
            lastpattern = p
            lastpatterns.append(lastpattern)
            break


In [317]:
# last marked delegates:
c = Counter(lastfragments)
# c.most_common():
#     print(i[0], i[1])
pd.DataFrame(c.most_common(), columns=['name', 'number'], )


,name,number
0,van der Lelij,254
1,du Tour,8
2,van Sloterdijck,7
3,Tamminga van Alberda,6
4,Rouse,5
5,Altingh,5
6,Polman,4
7,van Sminia,3
8,de Witt,2
9,Gockinga,2


In [318]:
for pers in Counter(lastfragments).most_common(10):
    print(pers, [goodvariants[d] for d in goodvariants if goodvariants[d]['match_name'].lower() == pers[0].lower()])
    #print(pers, [runner.all_matched[d] for d in runner.all_matched if runner.all_matched[d]['name'].lower() == pers[0].lower()])

('van der Lelij', 254) []
('du Tour', 8) [defaultdict(<class 'list'>, {'variant_pattern': 'du Tour', 'variant_score': 1.0, 'variant_keyword': 'du Tour', 'match_keyword': 'du', 'match_name': 'du Tour', 'match_score': 1.0})]
('van Sloterdijck', 7) [defaultdict(<class 'list'>, {'variant_pattern': ['advertent', 'advertentie'], 'variant_score': 1.0, 'variant_keyword': 'advertent', 'match_keyword': 'te', 'match_name': 'van Sloterdijck', 'match_score': 1.0})]
('Tamminga van Alberda', 6) [defaultdict(<class 'list'>, {'variant_pattern': ['Tamminga', 'Tammingu'], 'variant_score': 1.0, 'variant_keyword': 'Tamminga', 'match_keyword': 'Tamminga', 'match_name': 'Tamminga van Alberda', 'match_score': 1.0})]
('Rouse', 5) [defaultdict(<class 'list'>, {'variant_pattern': ['Kouse', 'Reuse', 'Rouse'], 'variant_score': 1.0, 'variant_keyword': 'Rouse', 'match_keyword': 'Rouse', 'match_name': 'Rouse', 'match_score': 1.0})]
('Altingh', 5) []
('Polman', 4) [defaultdict(<class 'list'>, {'variant_pattern': ['Pol

In [319]:
common_last_delegates = [pers[0] for pers in Counter(lastfragments).most_common(10)]
common_last_delegates

['van der Lelij',
 'du Tour',
 'van Sloterdijck',
 'Tamminga van Alberda',
 'Rouse',
 'Altingh',
 'Polman',
 'van Sminia',
 'de Witt',
 'Gockinga']

- TODO: search after last delegate for common other last delegates
- TODO: truncate text and spans after last known delegate.

## Presidents code

In [320]:
runner1715.presidents

['Velters',
 'Velters',
 'Velters',
 'Taets van Amerongen',
 'Taets van Amerongen',
 'Taets van Amerangen',
 'Taets van Amerongen',
 'Taets van Amerongen',
 'Taets van Amerongen',
 'van Burmania',
 'van Burmania',
 'van Burmania',
 'van Burmania',
 'van Burmania',
 'van Burmania',
 'van Isselmuden',
 'van Isselmuden',
 'van ssselmuden',
 'van lsselmuden',
 'van Isselmuden',
 'van ssselmuden',
 'Tamminga',
 'Tamminga',
 'Tamminga',
 'Tamminga',
 'Tamminga',
 'Tamminga',
 'van Welderen',
 'van Oldersom',
 'van Oldersom',
 'van Oldersom',
 'van Oldersom',
 'van Oldersom',
 'van Noortwyck',
 'van Noortwyck',
 'van Noortwyck',
 'van Noortwoyck',
 'van Noortwyck',
 'Fagel',
 'Velters',
 'Velters',
 'Velters',
 'Velters',
 'Velters',
 'Velters',
 'Taets van Amerongen',
 'Taets van Amerongen',
 'Taets van Amerongen',
 'Taets van Amerongen',
 'Taets van Amerongen',
 'Taets van Amerongen',
 'du Tour',
 'du Tour',
 'du Tour',
 'du Tour',
 'du Tour',
 'du Tour',
 'Sloet',
 'Sloet',
 'Sloet',
 'Slo

In [321]:
searchobs = runner1715.searchobs
met_en_zonder = {}
met_pr = {}
zonder_pr = {}
for T in searchobs:
    ob = searchobs[T].matched_text
    pr = [s for s in ob.spans if s.type == 'president']
    if len(pr)==0:
        zonder_pr[T]=pr
    else:
        met_pr[T]=pr
met_en_zonder['met_pr'] = met_pr
met_en_zonder['zonder_pr'] = zonder_pr
met_en_zonder

{'met_pr': {'session-1715-01-02-num-1': [fragment type president: 26-33],
  'session-1715-01-04-num-1': [fragment type president: 49-56],
  'session-1715-01-05-num-1': [fragment type president: 50-57],
  'session-1715-01-07-num-1': [fragment type president: 94-113],
  'session-1715-01-08-num-1': [fragment type president: 48-67],
  'session-1715-01-09-num-1': [fragment type president: 49-68],
  'session-1715-01-10-num-1': [fragment type president: 49-68],
  'session-1715-01-11-num-1': [fragment type president: 26-45],
  'session-1715-01-12-num-1': [fragment type president: 51-70],
  'session-1715-01-14-num-1': [fragment type president: 95-107],
  'session-1715-01-15-num-1': [fragment type president: 50-62],
  'session-1715-01-16-num-1': [fragment type president: 51-63],
  'session-1715-01-17-num-1': [fragment type president: 479-491],
  'session-1715-01-18-num-1': [fragment type president: 50-62],
  'session-1715-01-19-num-1': [fragment type president: 51-63],
  'session-1715-01-21-num-

Let's calculate some statistics for these presidents:


In [323]:
import statistics
starts = [v[0].begin for v in met_en_zonder['met_pr'].values()]
ends = [v[0].end for v in met_en_zonder['met_pr'].values()]
lengths = [v[0].end - v[0].begin for v in met_en_zonder['met_pr'].values()]
print("mean beginning and end offsets:")
meanstart = round(statistics.mean(starts),0)
print(meanstart)
meanend = round(statistics.mean(ends),0)
print(meanend)
print("min and max start offsets:")
print("min: ", min(starts),";max: ", max(starts))
print("min and max end offsets:")
print("min: ", min(ends), ";max: ", max(ends))
print("try and find some outlying values")
print("mean: ", round(statistics.mean(lengths), 0))
minlength = min(lengths)
maxlength = max(lengths)
minpres = [v[0].pattern for k,v in met_en_zonder['met_pr'].items() if v[0].end-v[0].begin==minlength ]
print("min: ", minlength , "max: ", maxlength)
#we should do something more with this, but at this point we cannot easily retrieve the spans


mean beginning and end offsets:
59.0
70.0
min and max start offsets:
min:  -1 ;max:  481
min and max end offsets:
min:  13 ;max:  491
try and find some outlying values
mean:  11.0
min:  5 max:  27


We compare try and complete the list of presidents by 1) finding the sessions without president and see if we can find any of them in the first part of the text. This

In [324]:
# this does not generalize much, but we'll leave that for later
from collections import defaultdict
from republic.analyser.attendance_lists.searchers import make_herensearcher
from republic.fuzzy.fuzzy_keyword_searcher import score_levenshtein_distance_ratio


class DelegateChecker(object):
    def __init__(self, keywords, searchobs):
        self.searcher = make_herensearcher(keywords=keywords)
        self.context_searcher = make_herensearcher(keywords={'PRAESENTIBUS':[], 'Den Heere':[]})
        self.searchobs = searchobs
        self.likely_prez = defaultdict(list)
        self.checkdeps = self.make_noprez()
        self.ks = self.make_keys()
        
    def make_keys(self):
        ks = list(self.checkdeps.keys())
        ks.sort() # we depend on the key order
        return ks
        
    def get_vicinity_presidents(self,t):
        mn = max(self.ks.index(t)-7, 0) 
        mx = min(self.ks.index(t)+7, len(ks)-1)
        likely_prez = defaultdict(list)
        for pt in ks[mn:mx]:
            if self.checkdeps[pt]:
                president = self.checkdeps[pt][0]
                name = president.delegate_name
                pattern = president.pattern
                likely_prez[name].append(pattern)
        return likely_prez
        # for p in likely_prez:
        #     ra.all_matched
#         xsisting_kws = list(self.searcher.keyword_index.keys())
#         kws = [k for k in list(likely_prez.keys()) if k not in xsisting_kws]            
#         self.searcher.index_keywords(kws)
#         for k in kws:
#             for variant in likely_prez[k]:
#                 self.searcher.index_spelling_variant(k, variant=variant)
    
    def check(self):
        results = []
        for t in self.ks:
            start = 0
            if not self.checkdeps[t]:
                txt = self.searchobs[t].text
                if len(txt) > 10:
                    res = self.searcher.find_candidates(text=txt)
                    if len(res) == 0:
                        break
                    hres = [r for r in res if r['match_keyword']=='Den Heere']
                    if hres:
                        sres = min(hres, key=lambda x: x['match_offset'])
                        if len(sres) > 0:
                            start = sres['match_offset']  + len(sres['match_string'])       
                    pres = [r for r in res if r['match_keyword']=='PRAESENTIBUS']
                    if len(pres) > 0:
                        eres = min(pres, key=lambda x: x['match_offset'])
                        end = eres['match_offset']
                    else:
                        end = start + 20
                    stxt = txt[start:end]
                    stxt = stxt.strip()
                likely_prez = self.get_vicinity_presidents(t)
                res = {}
                for cand in likely_prez:
                    candidate = likely_prez[cand]
                    for i in candidate:
                        res[i] = score_levenshtein_distance_ratio(i, stxt)
                chosen = max(res, key=lambda x: res[x])
                score = res[chosen]
                if score > 0.6:
#                     for cand in likely_prez:
#                         candidate = max([(i, score_levenshtein_distance_ratio(stxt, i)) for i in likely_prez[cand]])
                    candidate = chosen
                else:
                    prop = self.searcher.find_candidates(stxt)
                    if len(prop)>0:
                        candidate = max(prop, key=lambda x: x['levenshtein_distance'])
                    else:
                        candidate = None
                results.append({'txt': stxt,
#                                'lp':likely_prez,
                                'search result': candidate}) 
        return results

    
    def make_noprez(self):
        noprez = {}
        for T in self.searchobs:
            ob = self.searchobs[T]
            p = [s for s in ob.get_spans() if s.type=='president']
            if p:
                noprez[T] = p
            else:
                noprez[T] = None
        return noprez

In [325]:
s = make_herensearcher(keywords={'PRAESENTIBUS':[], 'Den Heere':[]})
res = [r for r in s.find_candidates(keyword="Den Heere", text="Den Heere Jan Steenbergen. PRAZSENTIBUS, De Heeren Van Lynden, van Singeudonck, van Heeckeren tot de") if r['match_keyword']=='Den Heere']
sres = min(res, key=lambda x: x['match_offset'])
sres['match_offset']  + len(sres['match_string'])

9

In [326]:
keywords = {i.name:[v.form for v in i.variants['general']] for i in runner1715.moregentlemen}


In [327]:
keywrds = {}
for e in runner1715.all_matched:
    i = runner1715.all_matched[e]
    nm = i["name"]
    f = [i["m_kw"]]
    f.extend([v[1] for v in i["variants"]])
    keywrds[nm]=list(set(f))
    

In [328]:
dlgchk = DelegateChecker(keywords=keywrds, searchobs=runner1715.searchobs)

In [329]:
dlgchk.searcher.find_candidates("Van Schwartzenbergh. ")

[]

In [330]:
max(res, key=lambda x: res[x])

TypeError: list indices must be integers or slices, not dict

In [331]:
dlgchk.check()

[{'txt': 'Martis den 1. Jannar', 'search result': None},
 {'txt': 'Dominica den 30. Jun', 'search result': None}]

In this case we only find 2 days without a president, so there is not much to remediate.

## Check delegates on their neighbours.

As explained above, delegates do not appear randomly in in the attendance lists, as these are dictated by the order of the provinces they represent. Because the group of delegates evolves over time and may vary any given day, there is no fixed order of delegates, but still we may assume often delegates have the same neighbours in the attendance list. Therefore we calculate the likely neighbours for delegates from the raw lists of spans. 

In [208]:
# let's create a sliding window for the spans of a day
from collections import deque

def window(seq, n=5):
    it = iter(seq)
    win = deque((next(it, None) for _ in range(n)), maxlen=n)
    yield win
    append = win.append
    for e in it:
        append(e)
        yield win

In [227]:
searchobs = runner1715.searchobs
wndws = []
for T in searchobs:
    spns = [(s.delegate_id, s.delegate_name, s.pattern) for s in ob.matched_text.spans if s.type in ['delegate', 'president'] and s.pattern is not '']
    wndws.append(list(list(x) for x in window(spns)))

In [228]:
wndws

[[[(17356, 'Tamminga van Alberda', 'Tamminga'),
   (19088, 'van Broeckhuijsen', 'Broeckhuysen,'),
   (18031, 'op ten Noorth', 'Noortiwyck,'),
   (14077, 'Heinsius', 'Heinsius.'),
   (17094, 'Velters', 'Velters,')],
  [(19088, 'van Broeckhuijsen', 'Broeckhuysen,'),
   (18031, 'op ten Noorth', 'Noortiwyck,'),
   (14077, 'Heinsius', 'Heinsius.'),
   (17094, 'Velters', 'Velters,'),
   (14024, 'van Wassenaer van Spanbroek', 'Spanbroeck,')],
  [(18031, 'op ten Noorth', 'Noortiwyck,'),
   (14077, 'Heinsius', 'Heinsius.'),
   (17094, 'Velters', 'Velters,'),
   (14024, 'van Wassenaer van Spanbroek', 'Spanbroeck,'),
   (16303, 'van Hoorn', 'Hoorn,')],
  [(14077, 'Heinsius', 'Heinsius.'),
   (17094, 'Velters', 'Velters,'),
   (14024, 'van Wassenaer van Spanbroek', 'Spanbroeck,'),
   (16303, 'van Hoorn', 'Hoorn,'),
   (17090, 'Coninck', 'Coninck.')],
  [(17094, 'Velters', 'Velters,'),
   (14024, 'van Wassenaer van Spanbroek', 'Spanbroeck,'),
   (16303, 'van Hoorn', 'Hoorn,'),
   (17090, 'Coninck',

In [287]:
from collections import defaultdict
z = []
matches = defaultdict(list)
for T in searchobs:
    ob = searchobs[T]
    for s in ob.matched_text.spans:
        if s.type in ['delegate', 'president'] and s.pattern == '':
            proposed_pattern = ob.text[s.begin:s.end]
            spl = re.split(r'[.,-]+', proposed_pattern)
            if len(spl)>1:
                proposed_pattern = spl[0]
            z.append(proposed_pattern)
            matches[proposed_pattern] = ob.text[s.begin:s.end]

In [299]:
from collections import defaultdict
z = []
matches = defaultdict(list)
for T in searchobs:
    ob = searchobs[T]
    for s in ob.matched_text.spans:
        if s.type in ['delegate', 'president'] and s.pattern == '':
            proposed_pattern = ob.text[s.begin:s.end]
            
            z.append(proposed_pattern)
            

In [ ]:
z

In [333]:
from fuzzy_search.fuzzy_phrase_searcher import FuzzyPhraseSearcher
from fuzzy_search.fuzzy_phrase_model import PhraseModel

# highger matching thresholds for higher quality OCR/HTR (higher precision, recall should be good anyway)
# lower matching thresholds for lower quality OCR/HTR (higher recall, as that's the main problem)
config = {
    "char_match_threshold": 0.8,
    "ngram_threshold": 0.6,
    "levenshtein_threshold": 0.8,
    "ignorecase": False,
    "ngram_size": 2,
    "skip_size": 2,
}

# initialize a new searcher instance with the config
fuzzy_searcher = FuzzyPhraseSearcher(config)

# create a list of domain keywords and phrases
#variants = [{'phrase': k, 'variants': v['variants']} for k, v in runner1715.all_matched.items()]
variants = []
for k, v in runner1715.all_matched.items():
    vrnts = [f[0] for f in v['variants']]
    vrnts.append(f"{v['variants'][0][0]}.DE")
    vrnt = {'phrase': v['name'],'variants':vrnts}
    variants.append(vrnt)
# create a PhraseModel object from the domain phrases
phrase_model = PhraseModel(model=variants,)
#phrase_model.add_phrases(ekwz.keys())
# phrase_model.add_variants(variants)
fuzzy_searcher.index_phrase_model(phrase_model=phrase_model)

In [346]:
phrase_model

TypeError: 'list' object is not callable

In [339]:
r = fuzzy_searcher.find_matches(text='Tamminga', include_variants=True,  use_word_boundaries=False)

In [336]:
result = {}
for i in list(set(z)):
    result[i] = fuzzy_searcher.find_matches(i, include_variants=True, use_word_boundaries=False)

In [338]:
f

In [367]:
str(match.variant.exact_string)

'nga'

In [368]:
for match in r:
    print(match.levenshtein_similarity, match.character_overlap, match.ngram_overlap, match.variant)
    print(score_levenshtein_distance_ratio(str(match.variant.exact_string),str(match.phrase.exact_string) ), str(match.variant.exact_string),str(match.phrase.exact_string))

1.0 1.0 1.0 Phrase(Tamminga, None)
0.36363636363636365 Tamminga Tamminga\ van\ Alberda
1.0 1.0 1.0 Phrase(minga, None)
0.3076923076923077 minga van\ Andringa
1.0 1.0 1.0 Phrase(inga, None)
0.3076923076923077 inga van\ Andringa
1.0 1.0 1.0 Phrase(nga, None)
0.23076923076923073 nga van\ Andringa


In [375]:
rom scipy.special import softmax
from numpy import argmax
def score_match(match):
    result  = sum([match.levenshtein_similarity, match.character_overlap, match.ngram_overlap, score_levenshtein_distance_ratio(str(match.variant.exact_string),str(match.phrase.exact_string))])
    return result

softscore = softmax([score_match(match) for match in r])
i = argmax(softscore)
r[i]

PhraseMatch(phrase: "Tamminga van Alberda", variant: "Tamminga",string: "Tamminga", offset: 0)

In [337]:
result

{'': [],
 'Marle.DE': [],
 'Heems,': [],
 'Marle.Gockinga,': [PhraseMatch(phrase: "Gockinga", variant: "Gockinga",string: "Gockinga", offset: 6),
  PhraseMatch(phrase: "Gockinga", variant: "Gockinga",string: "Gockinga", offset: 6),
  PhraseMatch(phrase: "van Andringa", variant: "inga",string: "inga", offset: 10),
  PhraseMatch(phrase: "van Andringa", variant: "nga",string: "nga", offset: 11)],
 'Cininck.': [PhraseMatch(phrase: "Coninck", variant: "Coninck",string: "Cininck", offset: 0)],
 'Burum.': [],
 'Steenbergen,': [],
 'Tamminga.DEP': [PhraseMatch(phrase: "Tamminga van Alberda", variant: "Tamminga",string: "Tamminga", offset: 0),
  PhraseMatch(phrase: "Tamminga van Alberda", variant: "Tamminga.DE",string: "Tamminga.DE", offset: 0),
  PhraseMatch(phrase: "van Andringa", variant: "minga",string: "minga", offset: 3),
  PhraseMatch(phrase: "van Andringa", variant: "inga",string: "inga", offset: 4),
  PhraseMatch(phrase: "van Andringa", variant: "nga",string: "nga", offset: 5),
  Phras

In [135]:
res = fuzzy_searcher.find_matches(text="Heer. dikke Heeren", include_variants=True, use_word_boundaries=False, allow_overlapping_matches=True)
for item in res:
    print(item.score_levenshtein_similarity())

0.8
1.0
0.8
0.8333333333333334
1.0
1.0
1.0


In [ ]:
matchsearch = FuzzySearcher(config=fuzzysearch_config)
phrase
kws = defaultdict(list)
matcher = {}
for key in all_matched:
    variants = all_matched[key].get('variants')
    keyword = all_matched[key].get('m_kw')
    idnr = all_matched[key].get('id')
    name = all_matched[key].get('name')
    for variant in variants:
        kws[keyword].append(variant[0])
        matcher[variant[0]] = {'kw':keyword, 'id':idnr, 'name':name}
matchsearch.index_keywords(list(kws.keys()))
for k in kws:
    for v in kws[k]:
        matchsearch.index_spelling_variant(keyword=k,variant=v)

In [241]:
for item in list(set(z)):
    print(runner1715.matchfnd.match_candidates(text=item))

TypeError: match_candidates() got an unexpected keyword argument 'text'

The match and span object takes the unmarked text from the objects and tries to match them with a delegate using the code for identifying delegates. But the failures in the ocr complicates this so that we have to use isolated strings, instead of splitting the text by commas or periods and newlines. This  drops all sorts of prefixes and in the case of a composite name such as _Taats van Amerongen_ it also either makes a name match twice or match part of it with another name. Also the prefixes like _van_ can get matched to the wrong names. So what we will do is match the names and then return to the list of spans and see if we cannot make a better match with the whole names. We would also like  to include previously acquired knowledge for better matching, but we have to think about that more.
Another thing is that we may overfit existing names, but we'll see about that.

In [69]:
from fuzzy_search.fuzzy_searcher import FuzzySearcher
from fuzzy_search.fuzzy_phrase_model import PhraseModel

config = {'char_match_threshold': 0.7,
 'ngram_threshold': 0.6,
 'levenshtein_threshold': 0.6,
 'ignorecase': False,
 'ngram_size': 2,
 'skip_size': 2}

sps = FuzzySearcher(config)

spns = [(s.pattern, s.get_delegate()) for s in ob.spans if s]
kws = []
for s in spns:
    nm = s[1]['name']
    d_id = s
#    if nm == 'Taets van Amerongen':

phrase_model = PhraseModel(model=variants, )
phrase_model.add_phrases(kws)
sps.index_phrase_model(phrase_model=phrase_model)

AttributeError: 'FuzzySearcher' object has no attribute 'keyword_index'

In [81]:
spns

[('', {'id': 0, 'name': '', 'score': 0}),
 ('', {'id': 0, 'name': '', 'score': 0}),
 ('', {'id': 0, 'name': '', 'score': 0}),
 ('', {'id': 0, 'name': '', 'score': 0})]

In [82]:
matchsearch = FuzzyPhraseSearcher(config=fuzzysearch_config)
kws = defaultdict(list)
matcher = {}
for key in all_matched:
    variants = all_matched[key].get('variants')
    keyword = all_matched[key].get('m_kw')
    idnr = all_matched[key].get('id')
    name = all_matched[key].get('name')
    for variant in variants:
        kws[keyword].append(variant[0])
        matcher[variant[0]] = {'kw':keyword, 'id':idnr, 'name':name}
matchsearch.index_keywords(list(kws.keys()))
for k in kws:
    for v in kws[k]:
        matchsearch.index_spelling_variant(keyword=k,variant=v)

In [207]:
deputyregister = defaultdict(list)
search_results = {}
fm = FndMatch(year=1728,
                 #patterns=pats,
                 register=register,
                 rev_graph=transposed_graph,
                 searcher=herensearcher,
                 junksearcher=junksweeper,
                 df=abbreviated_delegates)
for T in searchobs:
    ob = searchobs[T].matched_text
    mo = MatchAndSpan(ob, junksweeper=junksweeper, previously_matched=previously_matched, match_search=matchsearch)
    for span in ob.spans:
        if span.type in ['president', 'delegate']:
            if span.pattern == '':
                span.set_pattern(pattern=ob.item[span.begin:span.end])
            r = fm.match_candidates(heer=span.pattern)
            span.set_delegate(delegate_id=r.id, 
                              delegate_name=r.proposed_delegate, 
                              delegate_score=r.score)

In [93]:
matchfnd

## Continue matching

In [204]:
defo = defaultdict(list)
for item in ob.get_unmatched_text():
    item = [i.strip() for i in re.split('[\.|,]', item)]
    for i in item:
        identified = ''
        res = {}
        r2 = {}
        for g in moregentlemen:
            mn = 0
            co = 0
            mc = 0
            vrts = g.variants['general']
            dr = [score_levenshtein_distance_ratio(x.form, i) for x in vrts if x.heerid==g.heerid]
            if len(dr)>0:
                mn = mean(dr)
            res[g] = {'mn':mn}
            if len(i)<5:
                co = [score_ngram_overlap_ratio(x.form, i, 1) for x in vrts if x.heerid==g.heerid]
                if co and len(co)>0:
                  mc = mean(co)
            res[g]['mc'] = mc
        best_l = max(res, key=lambda key: res[key]['mn'])
        best_c = max(res, key=lambda key: res[key]['mc'])
        if res[best_l]['mn'] > 0.5:
            identified = res[best_l]['mn']
            if best_c != 0:
                if res[best_c] != identified:
                    identified = res[best_c]
        if identified != '':
            delegate = fm.match_candidates(heer=best_l.name)
            identified = delegate
        else: 
            identified = None
        if identified:
            
            name = delegate.proposed_delegate
            m_id = delegate.id
        else:
            name = ''
            m_id = 0
        start = ob.item.find(i)
        end = start + len(i)
        ob.set_span(span=(start, end),pattern=i, clas='deputy', delegate_name=name,
                    delegate_id=m_id)
#         defo[i]={'mn':(best_l,res[best_l]['mn']),
#                  'co':(best_c,res[best_c]['mc'])}
# defo

In [205]:
def get_mn(k, defo=defo):
    defo = dict(defo)
    v = defo[k]['mn'][1]
    v = float(v)
    return v
    
#         mn = dict(defo[k].get('mn').value
for k in defo:
    v = get_mn(k)
    if v < 0.5:
        res = iterative_search(name=k, year=1728, df=abbreviated_delegates)
        res

In [206]:
s = moregentlemen[8]
print(s.variants)
for v in s.variants:
    print(score_levenshtein_distance_ratio('an Schwartzenberg', s.name))
    #list(map(lambda x: score_levenshtein_distance_ratio('an Schwartzenberg', x), s.variants))

{'general': [V an S chwart zenbergh., Van Schwartzenbergh., Sehwartzenbergh. P, Han Schwartzenbergh., Van Sebwartzenbergh., an Schwartzenberg., Pan Schwartzenbergh., Van Schwartzenhergh., Van Sehwartzenbergh. P, Jan Schwartzenbergb., Van Schwartzenberg., Han Schwartzenberg., Van Schwartzenbergh.  P, Zan Schwartzenbergh., Van Schwartzenbergh. P, an Schwartzenbergh. P, an Schwart zenhergh., Schwartzenberg, Schwartzenberdh, zenbargh, zenborgl, Sebwartzenbergh, Schwartzenbersk, zenhorgh, Cattenboreh, Cartenborgh, Schwartzenberth, Cattenhorgh, Schwartzenberch, Cattenbergh, Schwattzenbergh, Schwartzenbergb, attenborgh, Schwartzenbereh, Schwartzenberghb, Zzenhorgh, Cáttenborch, Cattcnborgh, zenbergh, zenberghh, Cattenborgh, Cattenaorgh, tzenborghn, Caitenborgh, Sehwartzenbergh, Schwartzenbersh, Cattcnbergh, Schwartzènherch, Cattenborsh, Schwartvenbergh, Schwartenbergh, Cattenboroh, chwartzenbergh, Sehwartzenbergb, Schwartzenbersb, zenborah, tenboreh, Cattenbor, Zenborgh, zenboreh, Scbwartzenb

In [ ]:
v = s.variants['general'][0]
v.form

In [ ]:
res

In [120]:
span.pattern

'Tamminga.'

In [121]:
# span.set_delegate(delegate_id=10, 
#                               delegate_name='blop', 
#                               delegate_score=1.0)
span.delegate_id

17356

In [122]:
all_matched.keys()

dict_keys([13046, 16377, 18838, 16901, 17355, 17753, 16423, 15207, 12367, 16446, 13477, 17094, 18599, 17356, 17379, 16160, 17397, 16465, 15226, 19094, 12332, 19119, 13100, 2798, 12744, 19297, 14821, 15268, 13492, 13479, 19114, 18570, 17505, 16400, 14074, 17702, 19627, 19130, 19518, 15059, 16447, 12358, 13458, 17837, 17942, 19181, 14865, 19871, 19778, 17563, 17268, 16303, 453, 12537, 17635, 17927, 19123, 14047, 17340, 16492, 14079, 13401, 13911, 16491, 14573, 16479, 17090, 1451, 15265, 13384, 16404, 14697])

In [123]:
previously_matched

,Unnamed: 0,name,match_keyword,levenshtein_distance,proposed_delegate,id,score,probably_junk
0,7,Singendonck,Singendonck,1.000000,Singendonck,19094,1.0,False
1,8,Torck,Torck,1.000000,Torck,19114,1.0,False
2,9,Umbgroeven,Umbgrove,0.888889,Umbgrove,17268,1.0,False
3,11,Boetzelaar,Boetzelaar,1.000000,van Boetzelaar,17501,1.0,False
4,12,Steyn,Steyn,1.000000,Steyn,14009,1.0,False
5,13,Marseveen,Maarseveen,0.900000,Huydecoper van Maarseveen,13642,0.5,False
6,14,Zuylen,Zuylen,1.000000,van Zuylen van Nyvelt,18913,1.0,False
7,15,Nyvelt,Nyvelt,1.000000,van Zuylen van Nyvelt,18913,0.9,False
8,16,Boon,Boon,1.000000,Boon,15168,0.9,False
9,17,Velters,Velters,1.000000,Velters,17094,1.0,False


### Some Evaluations (fwiw)

In [124]:
pd.DataFrame(previously_matched.aggregate("mean")[['levenshtein_distance', 'score']])

,0
levenshtein_distance,0.807220
score,0.780645


In [125]:
previously_matched.score.value_counts()

1.0    19
0.0     5
0.9     3
0.6     2
0.8     1
0.5     1
Name: score, dtype: int64

In [126]:
previously_matched.levenshtein_distance.round(2).value_counts().sort_values('index', ascending=False)

1.00    19
0.00     5
0.89     3
0.80     2
0.86     1
0.90     1
Name: levenshtein_distance, dtype: int64

In [127]:
for s in ob.spans:
    print(s, s.idnr, ob.item[s.begin:s.end], s.get_delegate())

fragment type preamble: 0-30 1360 Veneris den 31. December 1728. {'id': 0, 'name': '', 'score': 0}
fragment type deputy: 0-0 8537  {'id': 0, 'name': '', 'score': 0}
fragment type deputy: 0-0 8539  {'id': 0, 'name': '', 'score': 0}
fragment type deputy: 0-0 8540  {'id': 0, 'name': '', 'score': 0}
fragment type deputy: 0-0 8541  {'id': 0, 'name': '', 'score': 0}
fragment type deputy: 0-0 8542  {'id': 0, 'name': '', 'score': 0}
fragment type deputy: 0-0 8543  {'id': 0, 'name': '', 'score': 0}
fragment type deputy: 0-0 8544  {'id': 0, 'name': '', 'score': 0}
fragment type deputy: 0-0 8545  {'id': 0, 'name': '', 'score': 0}
fragment type deputy: 0-0 8546  {'id': 0, 'name': '', 'score': 0}
fragment type deputy: 0-0 8547  {'id': 0, 'name': '', 'score': 0}
fragment type deputy: 0-0 8548  {'id': 0, 'name': '', 'score': 0}
fragment type deputy: 0-0 8549  {'id': 0, 'name': '', 'score': 0}
fragment type deputy: 0-0 8550  {'id': 0, 'name': '', 'score': 0}
fragment type deputy: 0-0 8551  {'id': 0, '

## a better searcher and identifier.

The problem: 
We have two ways of identifying name patterns now.

- the first uses pandas based method to identify strings against a database of known delegates and then pulls in some of the external context. This works pretty well for initial identification, but not all names are recognized very well. There is especially a problem by names that have been badly garbled by the text recognition, which is now the case with the OCR, but it will probably not get better if we start using HTR and look up entities in the text. Another issue is that this method has no memory of earlier work, both in the sense that it will not know if earlier matches were found or earlier proposals were marked as false. 
- the fuzzysearcher is very good for spotting name patterns in text as it includes approximate text matching and a number of variants. But it knows nothing about identification and there is no easy way of associating patterns with entities.
- fuzzysearcher is also able to cluster alike word patterns and recluster heterogeneous lists. However, this does not per se bring us closer to identification
- the only way to include previously selected variants is in the fuzzy searcher. Which is useful, but the main problem remains associating this with external knowledge.

This leads to the following requirements:
- a way of associating patterns with identities. But how. Earlier I put the result of a Fuzzysearch pattern clustering in a dataframe, but this included heterogeneous patterns. One possibility is to revise candidates and recategorize weak candidates. The goals is to incrementally improve the patterns associated with a name. This is especially useful if we reuse the objects over many years.
- another idea is to check for internal consistency of the results and try and find inconsistent results. This is especially useful for the presidents, as their number is even more limited than the delegates. The problem is overfitting in which unidentified names get associated with identified names and also new or infrequent delegates may drop out. SO there is a need for manual check.
- another tool may be to make a specialized confusion matrix for delegates

We start with this latest
no we don't. Most of this is already in place. The main problem is that we cannot access the it from 'both' sides, so that we have to rematch all over again. Which is what we do not want, because it slows down the process *and* it introduces mistakes all over again. What we need is memory:
- of previous matches
- of previous associations
- of previous wrong associations
and perhaps more

    

In [128]:
def get_delegates_from_spans(ob):
    result = {}
    for s in ob.spans:
        result['delegate'] = s.get_delegate()
        result['pattern'] = s.pattern
    return result

In [129]:
previously_matched.columns

Index(['Unnamed: 0', 'name', 'match_keyword', 'levenshtein_distance',
       'proposed_delegate', 'id', 'score', 'probably_junk'],
      dtype='object')

In [150]:
unclassified = []
nwheren = []
clean_gentlemen = []
for g in moregentlemen:
    matches = defaultdict(list)
    matchkw = g.matchkw
    matchkw = matchkw.strip()
    nwmatches = {}
    if matchkw == '':
        unclassified = g.variants['general'] # we go to the next instance and look into this later
        continue
    for v in g.variants['general']:
        matches[v.match].append(v)
        for key in matches.keys():
            name = g.name
            nwmatches[matchkw] = []
            s = score_levenshtein_distance_ratio(key, matchkw)
            if s > 0.7:
                nwmatches[matchkw].extend(matches[key])
            else:
                if not re.search('\w+', key):
                    #this is an empty key
                    meanscore = mean([score_ngram_overlap_ratio(kw.form, matchkw, 1) for kw in matches[key]])
                    if meanscore > 0.8:
                         nwmatches[matchkw].extend(matches[key])
        #            print('meanscore: ', key, meanscore)

                # is there no strange 'Van' variant in this?
                elif len(key)<5:
                    s2 = score_ngram_overlap_ratio(key, 'van', 1)
                    if s2 > 0.8:
                        meanscore = mean([score_levenshtein_distance_ratio(kw.form, matchkw, 1) for kw in matches[key]])
                        nwmatches[matchkw].extend(matches[key])
                        #print('meanscore: ', key, meanscore)
                    else: 
                        unclassified.extend(matches[key])
                else:
                    meanheerscore = mean([score_levenshtein_distance_ratio(kw.form, matchkw) for kw in matches[key]])
                    meankeyscore = mean([score_levenshtein_distance_ratio(kw.form, key) for kw in matches[key]])
                    meanheergramscore = mean([score_ngram_overlap_ratio(kw.form, matchkw, 2) for kw in matches[key]])
                    meankeygramscore = mean([score_ngram_overlap_ratio(kw.form, key, 2) for kw in matches[key]])
                    if meankeyscore>meanheerscore and meankeygramscore>meanheergramscore:
                        #print(key, meankeyscore-meanheerscore, meankeygramscore-meanheergramscore)
                        nwres = iterative_search(key, year=day, df=abbreviated_delegates)
                        rec = {'id': nwres.id.iat[0],
                               'm_kw': key,
                               'name': nwres.name.iat[0],
                               'variants': [(v.form, v.match, v.score) for v in matches[key]]}
                        nwheer = MatchHeer(rec=rec)
                        nwheren.append(nwheer)
    g.variants['new']=nwmatches
        
                
            

In [151]:
allvariants = Counter()
for g in moregentlemen:
    allvariants.update(g.variants['general'])
allvariants.most_common()

[(Bofeh, 18),
 (Taais, 16),
 (Taas, 15),
 (Zaats, 10),
 (7aats, 9),
 (aats, 7),
 (cock, 7),
 (Taals, 6),
 (Eelke, 6),
 (Pall, 6),
 (Danck-, 5),
 (Deck, 5),
 (Bers, 4),
 (Catten, 4),
 (Boftl, 3),
 (vats, 2),
 (Aan, 2),
 (ooch, 2),
 (Dack-, 2),
 (haaren, 2),
 (voor, 2),
 (Van Glinstra. P, 1),
 (Zan Giinftra., 1),
 (2an Glinftra., 1),
 (Jan Glinfra., 1),
 (Zan Glinftra., 1),
 (an Gliniira., 1),
 (Han Glinftra., 1),
 (Jan Glinftra, 1),
 (an Glinjtra., 1),
 (Van Glinstra., 1),
 (Van Glinftra., 1),
 (Van Glinflra., 1),
 (an Glinftra., 1),
 (Glnfra, 1),
 (Goflina, 1),
 (Goflinga, 1),
 (Goslinca, 1),
 (Glinsra, 1),
 (Glinfka, 1),
 (Glintra, 1),
 (Glnftra, 1),
 (Goslinga, 1),
 (Glnhra, 1),
 (CGlenstra, 1),
 (Clinftra, 1),
 (Ginftra, 1),
 (Glinfira, 1),
 (Glnlra, 1),
 (Glinfra, 1),
 (Glina, 1),
 (Glinftra, 1),
 (Clinfra, 1),
 (Glinstra, 1),
 (Gofliaga, 1),
 (Clinsra, 1),
 (Gilnftra, 1),
 (Glnlira, 1),
 (Gollinva, 1),
 (Glinhra, 1),
 (Gollinea, 1),
 (CGlinstra, 1),
 (Glînftra, 1),
 (Clinstra, 1),

ok, so we need something do discriminated them. Perhaps the time of their actual delegate appointments?
another idea is to try and match names more closely. But we do not implement that here (as I do not think there is much to gain here as yet)

In [153]:
clbm = DisambiguateDelegate()
clbm.closer_best_match(name='Aerssen', date="1628")

{'id': 38168,
 'exact': False,
 'row':   functienaam                                            college    regent  \
 1    griffier  Staten-Generaal der Verenigde Nederlanden (158...  Aerssens   
 
       id van van_als_bekend tot tot_als_bekend  persoon_id  functie_id  ...  \
 1  38168 NaT     28-09-1584 NaT        03-1627       17183         121  ...   
 
       old_lokaal old_provinciaal    old_regio    old_stand  \
 1  republiek_nan     republiek_0  republiek_0  republiek_0   
 
   old_vertegenwoordigend  periode       van_p       tot_p  \
 1            republiek_0        0  1584-09-28  1627-03-01   
 
                                             p_interv    daycompare  
 1  PeriodIndex(['1584-09-28', '1584-09-29', '1584...  <306 * Days>  
 
 [1 rows x 37 columns]}

# Consistency Checks
## Not finished

In [154]:
unmt = [searchobs[so].matched_text.get_unmatched_text() for so in searchobs]

In [155]:
wrds = [k[0].form for k in allvariants.most_common() if k[0] not in ['',None]]
fks.find_close_distance_keywords(wrds)


defaultdict(list,
            {'Bofeh': ['Bofih', 'Bofch', 'Bofth'],
             'Bofih': ['Bofeh', 'Bofib', 'Bofch', 'Bofth'],
             'Bofch': ['Bofeh', 'borch', 'Bofih', 'Bosch', 'Bofth'],
             'Bofth': ['Bofeh', 'Boftl', 'Bofih', 'Bofch'],
             'Taais': ['Taas', 'Taals', 'Taats'],
             'Taas': ['Taais', 'Taals', 'Taats'],
             'Taals': ['Taais', 'Taas', 'Taats'],
             'Taats': ['Taais',
              'Taas',
              'Zaats',
              '7aats',
              'aats',
              'Taals',
              'Tuats',
              'Táats',
              'Tauts',
              'Tagts',
              'Taáts'],
             'Zaats': ['7aats', 'aats', 'Taats'],
             '7aats': ['Zaats', 'aats', 'Taats'],
             'aats': ['Zaats', '7aats', 'vats', 'Taats'],
             'vats': ['aats', 'Vas', 'vas'],
             'cock': ['oock', 'Cocq', 'Coca', 'Cocg', 'Coco'],
             'oock': ['cock', 'ooch', 'Roock'],
             'Coc

In [156]:
for nwh in nwheren:
    if nwh.heerid not in all_matched.keys():
        print(nwh.heerid, nwh.name)

13050 van Goslinga
13050 van Goslinga
13050 van Goslinga
13050 van Goslinga
13050 van Goslinga
13050 van Goslinga
13050 van Goslinga
13050 van Goslinga
13050 van Goslinga
13050 van Goslinga
13050 van Goslinga
13050 van Goslinga
13050 van Goslinga
13050 van Goslinga
13050 van Goslinga
13050 van Goslinga
13050 van Goslinga
13050 van Goslinga
13050 van Goslinga
13050 van Goslinga
13050 van Goslinga
13050 van Goslinga
13050 van Goslinga
13050 van Goslinga
13050 van Goslinga
13050 van Goslinga
13050 van Goslinga
13050 van Goslinga
13050 van Goslinga
13050 van Goslinga
17862 van Welderen
17862 van Welderen
17862 van Welderen
17862 van Welderen
17862 van Welderen
17862 van Welderen
17862 van Welderen
17862 van Welderen
17862 van Welderen
17862 van Welderen
17862 van Welderen
17862 van Welderen
17862 van Welderen
16495 Westenberg
16495 Westenberg
16495 Westenberg
16495 Westenberg
16495 Westenberg
16495 Westenberg
16495 Westenberg
16495 Westenberg
16495 Westenberg
16495 Westenberg
16495 Westenb

In [157]:
def joinheer(h1, h2):
    if not h1.heerid == h2.heerid:
        raise ValueError('ids not equal')
    forms1 = set([v.form for form in h1.variants])
    forms2 = set([v.form for form in h2.variants])
    difference = forms2.difference(forms1)
    
    for form in difference:
        v1 = [v for v in h2.variants['general'] if v.form==form][0]
        h1.variants['general'].append(v1)
    return h1

In [158]:
nnwheren = []


In [159]:
idvals = set([n.heerid for n in nwheren])
clusterednwh = defaultdict(list)
for heerid in idvals:
    clusterednwh[heerid] = [nwh for nwh in nwheren if nwh.heerid == heerid]
    

In [160]:
from itertools import groupby
ds = {}
sh = sorted(nwheren, key=lambda x: x.heerid)
for k, g in groupby(sh, key=lambda x: x.heerid):
    ds[k] = list(g)
for k in ds:
    h1 = ds[k][0]
    for h2 in ds[k][1:]:
        h1 = joinheer(h1,h2)
    nnwheren.append(h1)

In [161]:
nnwheren[2].variants

{'general': [churman]}

In [162]:
unclassified

[voorle-,
 voorlede,
 Taais,
 Taais,
 Taas,
 Taais,
 Taas,
 Taais,
 Taas,
 Taais,
 Taas,
 Taais,
 Taas,
 Taais,
 Taas,
 Zaats,
 Taais,
 Taas,
 Zaats,
 7aats,
 Taais,
 Taas,
 Zaats,
 7aats,
 Taais,
 Taas,
 Zaats,
 7aats,
 aats,
 Taais,
 Taas,
 Zaats,
 7aats,
 aats,
 Taals,
 Taais,
 Taas,
 Zaats,
 7aats,
 aats,
 Taals,
 Taais,
 Taas,
 Zaats,
 7aats,
 aats,
 Taals,
 Taais,
 Taas,
 Zaats,
 7aats,
 aats,
 Taals,
 Taais,
 Taas,
 Zaats,
 7aats,
 aats,
 vats,
 Taals,
 Aan,
 Bers,
 Bers,
 Bers,
 Bofeh,
 Bofeh,
 Bofeh,
 Bofeh,
 Bofeh,
 Bofeh,
 Bofeh,
 Bofeh,
 Bofeh,
 Bofeh,
 Bofeh,
 Bofeh,
 Bofeh,
 Bofeh,
 Bofeh,
 Bofeh,
 Boftl,
 Bofeh,
 Boftl,
 Catten,
 Catten,
 Catten,
 cock,
 cock,
 cock,
 cock,
 cock,
 cock,
 ooch,
 Danck-,
 Danck-,
 Danck-,
 Danck-,
 Dack-,
 Deck,
 Deck,
 Deck,
 Deck,
 Eelke,
 Eelke,
 Eelke,
 Eelke,
 Eelke,
 haaren,
 voor,
 Pall,
 Pall,
 Pall,
 Pall,
 Pall]

In [163]:
unob = unclassified[0]
unob.heer


'Vegelin van Claerbergen'

In [164]:
c = Counter(unclassified)
c.most_common()

[(Bofeh, 17),
 (Taais, 15),
 (Taas, 14),
 (Zaats, 9),
 (7aats, 8),
 (aats, 6),
 (cock, 6),
 (Taals, 5),
 (Eelke, 5),
 (Pall, 5),
 (Danck-, 4),
 (Deck, 4),
 (Bers, 3),
 (Catten, 3),
 (Boftl, 2),
 (voorle-, 1),
 (voorlede, 1),
 (vats, 1),
 (Aan, 1),
 (ooch, 1),
 (Dack-, 1),
 (haaren, 1),
 (voor, 1)]

In [165]:
unclas = fks.find_close_distance_keywords([v.form for v in unclassified])
G_unclas = nx.Graph()
d_nodes = sorted(unclas)
for node in d_nodes:
    attached_nodes = unclas[node]
    G_unclas.add_node(node)
    for nod in attached_nodes:
        G_unclas.add_edge(node, nod)
connected_unclas = list(nx.connected_components(G_unclas))


In [166]:
connected_unclas

[{'7aats', 'Zaats', 'aats', 'vats'},
 {'Bers'},
 {'Bofeh'},
 {'Boftl'},
 {'Catten'},
 {'Dack-', 'Danck-'},
 {'Deck'},
 {'Eelke'},
 {'Pall'},
 {'Taais', 'Taals', 'Taas'},
 {'cock'},
 {'voorle-', 'voorlede'}]

In [167]:
iterative_search('Ten Brincke', year=1728, df=abbreviated_delegates)


,id,name,geboortejaar,sterfjaar,age_at_repr,colleges,functions,period,sg,was_gedeputeerde,p_interval,h_life,score
4913,16400,ten Brink,1690-01-01,1757-09-10,36.0,"Raad van Deventer, Staten-Generaal der Verenig...","lid, ordinaris gedeputeerde, gecommitteerde","PeriodIndex(['1716-01-01', '1716-01-02', '1716...",True,True,"[1716, 1757]","[1690, 1757]",0.8
6252,17925,van Borck,NaT,1720-03-22,NaN,"Raad van Groningen, Gedeputeerde Staten van Gr...","lid, gecommitteerde, ordinaris gedeputeerde","PeriodIndex(['1695-01-01', '1695-01-02', '1695...",True,True,"[1695, 1720]","[1651, 1720]",0.8


In [168]:
iterative_search('Eck', year=1728, df=abbreviated_delegates)

,id,name,geboortejaar,sterfjaar,age_at_repr,colleges,functions,period,sg,was_gedeputeerde,p_interval,h_life,score
7307,19133,van Eck tot Nergena,1697-01-01,1760-12-31,29.0,"Ridderschap van het Kwartier Veluwe, Staten-Ge...","lid, ordinaris gedeputeerde","PeriodIndex(['1720-01-01', '1720-01-02', '1720...",True,True,"[1720, 1760]","[1697, 1760]",1.0
7834,19858,van Eck,1682-01-01,1736-11-28,44.0,"Hof van Gelre en Zutphen (1588-1795), Staten-G...","griffier, raadsheer, ordinaris gedeputeerde","PeriodIndex(['1708-10-25', '1708-10-26', '1708...",True,True,"[1708, 1738]","[1682, 1736]",1.0


In [141]:
for x in connected_unclas:
    for i in x:
        if not re.search('[0-9]+',i):
            print (i)
#         ir = [v for v in unclassified if v.form == i]
#         for vir in ir:
#             print (vir.form, vir.heerid, '\n\n')
            result = herensearcher.find_candidates(i)
            if len(result) > 0:
                eindresult = max(result, key=lambda keyword: keyword['levenshtein_distance'])
                

Zaats
aats
vats
Catten
Danck-
Dack-
Deck
Eelke
Pall
Taals
Taais
Taas
ooch
voor
voorle-
voorlede


# Placeholder

In [173]:
def span_checker(ob):
    """prints all spans from a searchobject"""
    spans = ob.matched_text.spans
    for span in spans:
        txt = ob.matched_text.item[span.begin:span.end]
        span.set_pattern(txt)

        #kand = matchsearch.find_candidates(txt)[0]
        msk = framed_gtlm.vs.apply(lambda x: levenst_vals(x, txt))
        mres = framed_gtlm.loc[msk==msk.max()]
        if len(mres)>0:

    #         setattr(span, 'delegate_id', mres.uuid.iat[0])
    #         setattr(span, 'delegate_name', mres.name.iat[0])
        #print(kand, all_matched[kand])
            span.set_delegate(delegate_id=mres.ref_id.iat[0], delegate_name=mres.name.iat[0], delegate_score=msk.max())
        print(span.type, span.idnr, txt, span.delegate_id, span.delegate_name, span.delegate_score)


In [105]:
from republic.helper.similarity_match import score_levenshtein_distance
twijfelresults = []
for im in runner1715.all_matched:
    item = runner1715.all_matched[im]
    if score_levenshtein_distance(item['m_kw'], item['name'])<0.6:
        twijfelresults.append(im)
twijfelresults

[17090, 15232, 14340, 16399, 18456, 14482, 15059]

In [107]:
len(runner1715.all_matched) - len(twijfelresults)

46

In [109]:
runner1715.all_matched[17090]

{'id': 17090,
 'm_kw': 'Coninck',
 'score': 1.0,
 'name': 'Coninck',
 'variants': [('Coninck', 'Coninck', 1.0)]}

In [84]:
def match2pattern(match:dict, bestpattern='', year=0):
    goodresults = []
    quarantine = []
    variants = match['variants']
    if bestpattern == '':
            try:
                bestpattern = max(variants, key=lambda x: x[2])[0]
            except TypeError:
                print (variant)
                raise
    if match['score']>0.8:
        results = goodresults
    else:
        results = quarantine
    for variant in variants:
        if variant[2]>0.8:
            result = {}
            result['variant_pattern'] = variant[0]
            result['variant_score'] = variant[2]
            result['variant_keyword'] = bestpattern
            result['match_keyword']=match['m_kw']
            result['match_name'] = match['name']
            result['match_score'] = match['score']
            result['match_id'] = match['id']
            results.append(result)
    return goodresults, quarantine


In [85]:
year = 1730
indexkws = []
qindexkws = []
for item in runner1715.all_matched:
    matches = runner1715.all_matched[item]
    i_kws, q_kws = match2pattern(matches, year=year)
    indexkws.extend(i_kws)
    qindexkws.extend(q_kws)

In [86]:
from collections import defaultdict
goodvariants = {}
joined_good_variants = defaultdict(list)
unique_ids = list(set([i.get('match_id') for i in indexkws]))

for k in unique_ids:
    dd = defaultdict(list)
    ikws = [d for d in indexkws if d.get('match_id') == k]
    for d in ikws: # you can list as many input dicts as you want here
        for key, value in d.items():
            if key != 'match_id':
                dd[key].append(value)
    for kie, v in dd.items():
        v = list(set(v))
        if len(v) == 1:
            v = v[0]
        dd[kie] = v
    goodvariants[k] = dd

In [87]:
goodvariants

{15232: defaultdict(list,
             {'variant_pattern': 'Fagel',
              'variant_score': 1.0,
              'variant_keyword': 'Fagel',
              'match_keyword': 'Fagel',
              'match_name': 'Fagel',
              'match_score': 1.0}),
 14340: defaultdict(list,
             {'variant_pattern': ['Lestevenon', 'Lesievenon'],
              'variant_score': [0.9, 1.0],
              'variant_keyword': 'Lestevenon',
              'match_keyword': 'Lestevenon',
              'match_name': 'Lestevenon',
              'match_score': 1.0}),
 17927: defaultdict(list,
             {'variant_pattern': ['Polmen', 'Polman'],
              'variant_score': 1.0,
              'variant_keyword': 'Polmen',
              'match_keyword': 'Pol',
              'match_name': 'Polman',
              'match_score': 1.0}),
 16399: defaultdict(list,
             {'variant_pattern': ['Kouse', 'Reuse', 'Rouse'],
              'variant_score': 1.0,
              'variant_keyword': 'Rouse',
 

In [88]:
proposed_good_matches = []
for item in qindexkws:
    if item['variant_score'] > item['match_score']:
        item['match_keyword'] = item['variant_pattern']
        item['match_score'] = item['variant_score']
        proposed_good_matches.append(item)
        qindexkws.pop(qindexkws.index(item)) 
        

In [89]:
vrs = [i['variant_pattern'] for i in indexkws]
vids = [i['match_id'] for i in indexkws]
prids = [i['match_id'] for i in proposed_good_matches]
vids.extend(prids)

In [90]:
max(president, key=lambda x: len(x['match_string']))

NameError: name 'president' is not defined

In [91]:
max(president, key=lambda x: x['levenshtein_distance'])

NameError: name 'president' is not defined

In [92]:
for item in qindexkws:
    if item['match_id'] in vids:
        print(item)

{'variant_pattern': 'Taets van Amerangen', 'variant_score': 1.0, 'variant_keyword': 'Taets van Amerongen', 'match_keyword': 'Taets', 'match_name': 'Cats', 'match_score': 0.7, 'match_id': 14074}
{'variant_pattern': 'Uingh', 'variant_score': 1.0, 'variant_keyword': 'Ubingh', 'match_keyword': 'Ubingh', 'match_name': 'Altingh', 'match_score': 0.7, 'match_id': 18369}
{'variant_pattern': 'Ubingh', 'variant_score': 1.0, 'variant_keyword': 'Ubingh', 'match_keyword': 'Ubingh', 'match_name': 'Altingh', 'match_score': 0.7, 'match_id': 18369}
{'variant_pattern': 'van Heuckelom', 'variant_score': 1.0, 'variant_keyword': 'van Heuckelom', 'match_keyword': 'Heuck', 'match_name': 'Eeck', 'match_score': 0.8, 'match_id': 17327}
{'variant_pattern': 'Heuckesom', 'variant_score': 1.0, 'variant_keyword': 'van Heuckelom', 'match_keyword': 'Heuck', 'match_name': 'Eeck', 'match_score': 0.8, 'match_id': 17327}
{'variant_pattern': 'Heuckes', 'variant_score': 1.0, 'variant_keyword': 'van Heuckelom', 'match_keyword

In [93]:
len(qindexkws)

28

In [94]:
len(proposed_good_matches)

28

In [95]:
from statistics import mean
mean([i.get('match_score') for i in indexkws])

0.9798165137614679

In [96]:
min([i['match_score'] for i in indexkws])

0.9

In [98]:
from republic.analyser.attendance_lists.identify import iterative_search
reslt = {}
for i in twijfelresults:
    item = runner.all_matched[i]
    l = 0
    v = ''
    v = max(item['variants'], key=lambda z: len(z[0]))[0]
    reslt[v] = iterative_search(name=v, year=1728, debug=False, df=runner.abbreviated_delegates)

NameError: name 'twijfelresults' is not defined

In [55]:
from republic.republic_keyword_matcher.finders.identify import identify

In [56]:
iterative_search(name='Taats', year=1728, df=runner.abbreviated_delegates)

,id,name,geboortejaar,sterfjaar,age_at_repr,colleges,functions,period,sg,was_gedeputeerde,p_interval,h_life,score
127,466,Taay,1753-01-01,1799-12-31,NaN,Staten-Generaal der Verenigde Nederlandse Prov...,"gedeputeerde, lid","PeriodIndex(['1795-02-04', '1795-02-05', '1795...",True,True,"[1795, 1796]","[1753, 1799]",0.7
2629,14074,Cats,1577-01-01,1660-12-31,NaN,Gecommitteerde Raden van Holland in het Zuider...,"raadpensionaris, ordinaris gedeputeerde","PeriodIndex(['1629-09-19', '1629-09-20', '1629...",True,True,"[1629, 1651]","[1577, 1660]",0.7


In [57]:
from IPython.core.display import display, HTML
for i in reslt:
    display(i, reslt[i])

'Zaats van Amorongen'

,id,name,geboortejaar,sterfjaar,age_at_repr,colleges,functions,period,sg,was_gedeputeerde,p_interval,h_life,score
144,483,Staal van Ingen,1755-01-01,1804-12-31,NaN,Staten-Generaal der Verenigde Nederlandse Prov...,gedeputeerde,"PeriodIndex(['1796-02-04', '1796-02-05', '1796...",True,True,"[1796, 1796]","[1755, 1804]",0.7
6161,17821,Borre van Amerongen,None,1640-01-21,NaN,"Ridderschap van Utrecht, Staten-Generaal der V...","lid, ordinaris gedeputeerde, gecommitteerde","PeriodIndex(['1606-01-01', '1606-01-02', '1606...",True,True,"[1606, 1640]","[1562, 1640]",0.7


'Van lsselmuden'

,id,name,geboortejaar,sterfjaar,age_at_repr,colleges,functions,period,sg,was_gedeputeerde,p_interval,h_life,score
7281,19091,van Essen Essen,None,None,NaN,Staten-Generaal der Verenigde Nederlanden (158...,ordinaris gedeputeerde,"PeriodIndex(['1713-05-01', '1713-05-02', '1713...",True,True,"[1713, 1716]","[1669, 1738]",0.7


'Jan MWiassenaer'

...

,id,name,geboortejaar,sterfjaar,age_at_repr,colleges,functions,period,sg,was_gedeputeerde,p_interval,h_life,score
1260,12531,Velsius,None,None,NaN,"Raad van Leeuwarden, Gedeputeerde Staten van F...","lid, ordinaris gedeputeerde, gecommitteerde","PeriodIndex(['1610-01-01', '1610-01-02', '1610...",True,True,"[1610, 1615]","[1566, 1637]",0.7
5793,17347,Pels,None,1705-06-22,NaN,Staten-Generaal der Verenigde Nederlanden (158...,"ordinaris gedeputeerde, extraordinaris gecommi...","PeriodIndex(['1680-02-24', '1680-02-25', '1680...",True,True,"[1680, 1705]","[1636, 1705]",0.7
7267,19074,Pels,None,None,NaN,Staten-Generaal der Verenigde Nederlanden (158...,ordinaris gedeputeerde,"PeriodIndex(['1711-05-01', '1711-05-02', '1711...",True,True,"[1711, 1715]","[1667, 1737]",0.7


'uytstel'

,id,name,geboortejaar,sterfjaar,age_at_repr,colleges,functions,period,sg,was_gedeputeerde,p_interval,h_life,score
6480,18200,Buttel,None,None,NaN,"Gedeputeerde Staten van Groningen (1595-1795),...","lid, gecommitteerde","PeriodIndex(['1598-03-02', '1598-03-03', '1598...",False,False,"[1598, 1602]","[1564, 1624]",0.3


'verhaalen'

,id,name,geboortejaar,sterfjaar,age_at_repr,colleges,functions,period,sg,was_gedeputeerde,p_interval,h_life,score
2600,14044,Vermeulen,1732-01-01,1810-06-12,NaN,"Vroedschap van Haarlem, Staten-Generaal der Ve...","lid, ordinaris gedeputeerde, gecommitteerde","PeriodIndex(['1761-01-01', '1761-01-02', '1761...",True,True,"[1761, 1795]","[1732, 1810]",0.7


G_heren = nx.Graph()
d_nodes = sorted(cl_heren)
for node in d_nodes:
    attached_nodes = cl_heren[node]
    G_heren.add_node(node)
    for nod in attached_nodes:
        G_heren.add_edge(node, nod)
pm_heren = list(nx.connected_components(G_heren))
pm_heren

In [58]:
item

{'id': 15267,
 'm_kw': 'Overhagen',
 'score': 0.8,
 'name': 'Schagen',
 'variants': [('verhaalen', 'Overhagen', 0.6666666666666667),
  ('verRaaden', '', 0)]}

In [174]:
from republic.fuzzy.fuzzy_keyword_searcher import FuzzyPhraseSearcher
fuzzysearch_config = {
    "char_match_threshold": 0.8,
    "ngram_threshold": 0.6,
    "levenshtein_threshold": 0.5,
    "ignorecase": False,
    "ngram_size": 2,
    "skip_size": 2,
}
fks = FuzzyPhraseSearcher(config=fuzzysearch_config)
fks.use_word_boundaries = False
kws = [goodvariants[p]['match_keyword'] for p in goodvariants]
false_kws = ['PRAESIDE','PRAESENTIBUS', 'Nihil Actum']
kws.extend(false_kws)
fks.index_keywords(kws)
for kw in kws:
    if kw not in false_kws:
        z = [goodvariants[d] for d in goodvariants if goodvariants[d]['match_keyword'] == kw][0]
        vrs = z["variant_pattern"]
        for v in vrs:
            fks.index_spelling_variant(kw, v)

In [181]:
suspect_obs = defaultdict(list)
for ob in runner.searchobs:
    fragments = runner.searchobs[ob].get_fragments()
    fragments.reverse()
    for item in fragments:
        lastdelegate = ''
        ld = item.get('delegate_name')
        if ld in common_last_delegates:
            break
        elif ld == '': 
            p = item.get('pattern')
            r = fks.find_candidates(text=p)
            if len(r)>0:
                r = max(r, key=lambda x: x['levenshtein_distance'])
                if r["match_keyword"] in false_kws:
                    suspect_obs[r["match_keyword"]].append(ob)
                elif r["match_keyword"] in commonlastgoodvariantsvs:
                    begin = item['begin']
                    changeob = runner.searchobs[ob]
                    begin = res[-2]
                    for span in changeob.matched_text.spans:
                        if span.begin>=begin:
                            changeob.matched_text.spans.remove(span)

In [182]:
suspect_obs

defaultdict(list,
            {'PRAESENTIBUS': ['session-1728-01-26-num-1',
              'session-1728-01-27-num-1',
              'session-1728-01-31-num-1',
              'session-1728-02-02-num-1',
              'session-1728-02-05-num-1',
              'session-1728-02-17-num-1',
              'session-1728-03-02-num-1',
              'session-1728-03-10-num-1',
              'session-1728-03-13-num-1',
              'session-1728-05-15-num-1',
              'session-1728-06-30-num-1',
              'session-1728-08-16-num-1',
              'session-1728-08-30-num-1',
              'session-1728-12-27-num-1'],
             'PRAESIDE': ['session-1728-02-17-num-1',
              'session-1728-08-30-num-1'],
             'Nihil Actum': ['session-1728-08-30-num-1']})

In [179]:
for i in suspect_obs:
    print(runner.searchobs[i].get_fragments())

[{'type': 'unmarked', 'pattern': '1728. ', 'end': 0, 'begin': 5}, {'type': 'delegate', 'idnr': 0, 'pattern': 'kan', 'begin': 6, 'end': 9, 'delegate_id': 13287, 'delegate_name': 'van Canter', 'delegate_gewest': '', 'delegate_score': 0.0}, {'type': 'unmarked', 'pattern': ' werden getreden. ', 'end': 9, 'begin': 26}, {'type': 'pre', 'idnr': 0, 'pattern': 'PRAESIDE, Den Heere', 'begin': 27, 'end': 46, 'delegate_id': 0, 'delegate_name': '', 'delegate_gewest': 0, 'delegate_score': 0}, {'type': 'president', 'idnr': 0, 'pattern': 'Van Wassenaer', 'begin': 47, 'end': 60, 'delegate_id': 13477, 'delegate_name': 'van Wassenaer van Obdam', 'delegate_gewest': '', 'delegate_score': 0.0}, {'type': 'unmarked', 'pattern': '. ', 'end': 60, 'begin': 61}, {'type': 'presentibus', 'idnr': 0, 'pattern': 'PRAESENTIBUS', 'begin': 62, 'end': 74, 'delegate_id': 0, 'delegate_name': '', 'delegate_gewest': 0, 'delegate_score': 0}, {'type': 'unmarked', 'pattern': ', De Heeren Van Lynden, van ', 'end': 74, 'begin': 10

In [169]:
begin = res[-2]
for span in changeob.matched_text.spans:
    if span.begin>=begin:
        changeob.matched_text.spans.remove(span)
changeob.matched_text.spans

[fragment type preamble: 0-31,
 fragment type pre: 32-51,
 fragment type president: 52-71,
 fragment type presentibus: 73-85,
 fragment type delegate: 113-122,
 fragment type delegate: 131-146,
 fragment type delegate: 163-173,
 fragment type delegate: 178-188,
 fragment type delegate: 198-205,
 fragment type delegate: 224-233,
 fragment type delegate: 255-267,
 fragment type delegate: 272-282,
 fragment type delegate: 283-288,
 fragment type delegate: 289-294,
 fragment type delegate: 316-326,
 fragment type delegate: 331-341,
 fragment type delegate: 346-355,
 fragment type delegate: 360-371,
 fragment type delegate: 376-384]

In [152]:
res[2]

{'match_keyword': 'Tamminga',
 'match_term': 'Tamminga',
 'match_string': 'Tamminga',
 'match_offset': 0,
 'char_match': 1.0,
 'ngram_match': 1.0,
 'levenshtein_distance': 1.0}

In [121]:
commonlastgoodvariants = {v:goodvariants[v] for v in goodvariants if goodvariants[v]['match_name'] in common_last_delegates}

In [122]:
commonlastgoodvariantsvs = []
for v in goodvariants:
    if goodvariants[v]['match_name'] in common_last_delegates:
        commonlastgoodvariantsvs.extend(goodvariants[v]['variant_pattern'])
commonlastgoodvariantsvs

['Polmen',
 'Polman',
 'Kouse',
 'Reuse',
 'Rouse',
 'Gockinga',
 'Gockingb',
 'van Sminia',
 'Sminia',
 'Sminta',
 'Smiria',
 'advertent',
 'advertentie',
 'Tamminga',
 'Tammingu',
 'd',
 'u',
 ' ',
 'T',
 'o',
 'u',
 'r']

In [123]:
lastspans = [f[-1].to_json() for f in out]

NameError: name 'out' is not defined

In [124]:
lastfragments[-100]a

SyntaxError: invalid syntax (<ipython-input-124-85e5f3e84b97>, line 1)

In [23]:
spans = []
for T in ra.searchobs:
    ob = ra.searchobs[T]
    spans.append(ob.get_spans())

In [125]:
noprez = {}
for T in runner1715.searchobs:
    ob = runner1715.searchobs[T]
    p = [s for s in ob.get_spans() if s.type=='president']
    if p:
        noprez[T] = p
    else:
        noprez[T] = None


In [127]:
upr = list(set(runner1715.presidents))
upr

['Tamminga',
 'van Renswoude',
 'van Steenbergen',
 'van Heuckelom',
 'van Hoorn',
 'Velters',
 'Coninck',
 'de Neyn',
 'Taets van Amerangen',
 'Rouse',
 'van Uterwyck tot Alerdinck',
 'Sloet',
 'van Oldersom',
 'Fagel',
 'van Burmania I',
 'van Noortwoyck',
 'van Isselmuden',
 'Taets van Amerongen',
 'van Ophemert',
 'van Uterwyck toe Alerdinck',
 'van Sminia',
 'Ubingh',
 'Vesters',
 'van Uterwycek tot Alerdinck',
 'Lesievenon',
 'du Tour',
 'van Burmania',
 'oan Lith de Jeude',
 'Uingh',
 'van lsselmuden',
 'Lestevenon',
 'van Welderen',
 'van Marle',
 'van Burum',
 'van Spanbroeck',
 'van Noortwyck',
 'van ssselmuden',
 'van Lith de Jeude']

In [128]:
ks = list(noprez.keys())
ks.sort()


In [129]:
noprez[t][0]

NameError: name 't' is not defined

In [106]:
fks.keyword_index.keys()

dict_keys(['Bout', 'de Drews', 'Borre van Amerongen', 'van Borssele', 'van Isselmuden tot Zwollingerkamp', 'van Lynden', 'van Wassenaer van Obdam', 'van Rechteren tot Gramsbergen', 'thoe Schwartzenberg en Hohenlansberg', 'ten Brink', 'Lohman', 'Nobel', 'Quintus', 'van Glinstra', 'van Haersolte tot Oldenhof', 'van Reede tot Renswoude', 'Singendonck', 'Tamminga van Alberda', 'van Welderen', 'Velters', 'van Wijnbergen tot de Glinthorst', 'Westenberg', 'Vegelin van Claerbergen', 'Weveringh', 'de Mepsche', 'Banga', 'van Heeckeren tot Barlham', 'van Deurn', 'Brandwijk van Blokland', 'van den Boetzelaar', 'Musch', 'van Heeckeren tot Brandsenburg', 'Fontein', 'Groeninx', 'Cock', 'Coulman', 'van der Dussen', 'Eelbo', 'van Maneel', 'van den Eede', 'Hagen', 'Verkerk', 'van Goslinga', 'Greven', 'Cramer', 'de Groot', 'van Heeckeren', 'Hendrickx', 'van Hoeckelom', 'Hoogerbeets', 'van Borssele van der Hooghe', 'van Huss tot Rees', 'Sloet tot Lindenhorst en Slotenhagen', 'van der Jagt', 'Torck', 'van 

In [190]:
len("'Van Lynden, van Singendonck, IS ter Vergaderinge gelesen de Requeste van Johan Coenraadts Hendricks")

100

In [191]:
score_levenshtein_distance_ratio('Van Schwartzenbergh', 'Van Schwart zenbergh')

0.95

In [192]:

fks = make_herensearcher(keywords={i.name:[v.form for v in i.variants['general']] for i in ra.moregentlemen})

NameError: name 'ra' is not defined

In [ ]:
make_herensearcher(keywords={})

In [83]:
for t in ks:
    if not noprez[t]:
        likely_prez = get_vicinity_presidents(noprez, t, )
        txt = ra.searchobs[t].text
        if len(txt) > 10:
            e = txt.find('PRAESENTIBUS')
            if e > -1:
                stxt = txt[:e]
            else: 
                stxt = txt[:100]
            print(likely_prez.find_candidates(stxt), stxt)
                
         

[{'match_keyword': 'Velters', 'match_term': 'Velters', 'match_string': 'Vesters', 'match_offset': 10, 'char_match': 0.8571428571428571, 'ngram_match': 0.75, 'levenshtein_distance': 0.8571428571428572}] Den Heere Vesters. 
[{'match_keyword': 'Velters', 'match_term': 'Velters', 'match_string': 'Velters', 'match_offset': 10, 'char_match': 1.0, 'ngram_match': 1.0, 'levenshtein_distance': 1.0}] Den Heere Velters. 
[{'match_keyword': 'van Reede tot Renswoude', 'match_term': 'van Reede tot Renswoude', 'match_string': 'Den Heere Van Renswoude', 'match_offset': 0, 'char_match': 0.7391304347826086, 'ngram_match': 0.5833333333333334, 'levenshtein_distance': 0.6956521739130435}] Den Heere Van Renswoude. 
[] Den Heere Van Schwartzenbergh. 
[] Den Heere Van Tamminga. 
[{'match_keyword': 'van Lynden', 'match_term': 'van Lynden', 'match_string': 'Van Lynden', 'match_offset': 10, 'char_match': 0.9, 'ngram_match': 0.8181818181818182, 'levenshtein_distance': 0.9}] Den Heere Van Lynden. 
[] Den Heere Zaat

In [65]:
run_attendancelist.get_delegates_from_spans(searchobs[T].matched_text)

{'delegate': {'id': 0, 'name': '', 'score': 0}, 'pattern': ''}

In [ ]:
for k in ra.searchobs:
    so = ra.searchobs[k]
    sspans = so.get_spans()
    for s in sspans:
        if s.pattern in s.delegate_name:
            if s.delegate_name > s.pattern
            so.get_txt

In [ ]:
ra.all_matched

# OUTPUT

In [62]:
searchobs = runner1715.searchobs

In [63]:
from IPython.display import HTML
htmlout=[]
for T in searchobs:
    ob = searchobs[T].matched_text
    url = searchobs[T].make_url()
    ob.mapcolors()
    rest = ob.serialize()
    rest = f"\n<h4>{T}</h4>\n" + rest
    if url:
        rest += f"""<br/><br/><a href='{url}'>link naar {T}-image</a><br/>"""
    htmlout.append(rest)
#out.reverse()
HTML("<br><hr><br>".join(htmlout))


In [27]:

for T in searchobs:
            print(searchobs[T].resumptionstart, searchobs[T].text)


0 Martis den 1. Jannarii1715.Mercurii den 2. JanuariiDen Heere Velters.Broeckhuysen, Ham.landt.Zeeland. PRAESENTIBUS, De Heeren van Oldersm, van Essen, vanVan Noertwyck, Fagel, Fabricius, van Castricum, Raedtpensionaris Heinsius, met DEvan deDN,NNEDENENBNDEDNB U)DHEERENNOEENODEEDDEDENSSEYderVERBENIGRDENREDERLANDEN.De anno 1715.
303 1715. PRAESIDE, Den Heere Velters. PRAESENTIBUS, De Heeren van Oldersom, van Essen, van Broeckhuysen, Ham. Van Noortwyck, Fagel, van Alphen, Raedtpensionaris Heinsius. Van Spanbroeck , Coninck. Taets van Amerongen, van Renswoude. Du Tour. Isselmuyden , Rouse. Tamminga,DE Resorutien , gisteren genomen, zyn gelesen ende geresumeert, gelijck oock geresumeert ende gearresteert zyn de depesches daer uyt resulterende.
468 Veneris den 4. Januarii1714. PRAESIDE, Den Heere Velters. PRAESENTIBUS, De Heeren van Oldersom, van Essen, van Broeckhuysen, Ham. Van Noortwyck, Fagel , Fabricius, van Alphen, Hulshout, Raedtpensionaris Heinsius, met een extraordinaris Gedeputeer